In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Discrete Variables

There may be times when it’s necessary to pass variables that are not floats or float arrays between components. These variables can be declared as discrete variables. A discrete variable can be any picklable python object.

In explicit and implicit components, the user must call `add_discrete_input` and `add_discrete_output` to declare discrete variables in the `setup` method.

## Methods for Adding Discrete Variables

Here are the methods used to add discrete variables to components.

```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_discrete_input
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_discrete_output
        :noindex:
```

## Discrete Variable Considerations

Discrete variables, like continuous ones, can be connected to each other using the `connect` function or by promoting an input and an output to the same name. The type of the output must be a valid subclass of the type of the input or the connection will raise an exception.

```{warning}
If a model computes derivatives and any of those derivatives depend on the value of a discrete output variable, an exception will be raised.
```

If a component or group contains discrete variables, then the discrete inputs and/or outputs will be passed to the relevant API functions. In general, if nonlinear inputs are passed to a function, then a discrete inputs argument will be added. If nonlinear outputs are passed, then a discrete outputs argument will be added. The signatures of the affected functions are shown below:

```{eval-rst}
    .. automethod:: openmdao.core.explicitcomponent.ExplicitComponent.compute
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.explicitcomponent.ExplicitComponent.compute_jacvec_product
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.explicitcomponent.ExplicitComponent.compute_partials
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.implicitcomponent.ImplicitComponent.apply_nonlinear
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.implicitcomponent.ImplicitComponent.guess_nonlinear
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.implicitcomponent.ImplicitComponent.linearize
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.group.Group.guess_nonlinear
        :noindex:
```

## Discrete Variable Examples

An example is given below that shows an explicit component that has a discrete input along with continuous inputs and outputs.

In [ ]:
import numpy as np
import openmdao.api as om


class BladeSolidity(om.ExplicitComponent):
    def setup(self):

        # Continuous Inputs
        self.add_input('r_m', 1.0, units="ft", desc="Mean radius")
        self.add_input('chord', 1.0, units="ft", desc="Chord length")

        # Discrete Inputs
        self.add_discrete_input('num_blades', 2, desc="Number of blades")

        # Continuous Outputs
        self.add_output('blade_solidity', 0.0, desc="Blade solidity")

    def compute(self, inputs, outputs, discrete_inputs, discrete_outputs):

        num_blades = discrete_inputs['num_blades']
        chord = inputs['chord']
        r_m = inputs['r_m']

        outputs['blade_solidity'] = chord / (2.0 * np.pi * r_m / num_blades)

# build the model
prob = om.Problem()

prob.model.add_subsystem('SolidityComp', BladeSolidity(),
                         promotes_inputs=['r_m', 'chord', 'num_blades'])

prob.setup()

prob.set_val('num_blades', 2)
prob.set_val('r_m', 3.2)
prob.set_val('chord', .3)

prob.run_model()

# minimum value
print(prob['SolidityComp.blade_solidity'])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob['SolidityComp.blade_solidity'], 0.02984155, 1e-4)

Similarly, discrete variables can be added to implicit components.

In [ ]:
import openmdao.api as om

class ImpWithInitial(om.ImplicitComponent):
    """
    An implicit component to solve the quadratic equation: x^2 - 4x + 3
    (solutions at x=1 and x=3)
    """
    def setup(self):
        self.add_input('a', val=1.)
        self.add_input('b', val=-4.)
        self.add_discrete_input('c', val=3)
        self.add_output('x', val=5.)

        self.declare_partials(of='*', wrt='*')

    def apply_nonlinear(self, inputs, outputs, residuals, discrete_inputs, discrete_outputs):
        a = inputs['a']
        b = inputs['b']
        c = discrete_inputs['c']
        x = outputs['x']
        residuals['x'] = a * x ** 2 + b * x + c

    def linearize(self, inputs, outputs, partials, discrete_inputs, discrete_outputs):
        a = inputs['a']
        b = inputs['b']
        x = outputs['x']

        partials['x', 'a'] = x ** 2
        partials['x', 'b'] = x
        partials['x', 'x'] = 2 * a * x + b

    def guess_nonlinear(self, inputs, outputs, resids, discrete_inputs, discrete_outputs):
        # Default initial state of zero for x takes us to x=1 solution.
        # Here we set it to a value that will take us to the x=3 solution.
        outputs['x'] = 5

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', ImpWithInitial())

model.comp.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.comp.linear_solver = om.ScipyKrylov()

prob.setup()
prob.run_model()

In [ ]:
print(prob.get_val('comp.x'))

In [ ]:
assert_near_equal(prob.get_val('comp.x'), 3., 1e-4)